In [1]:
import os
from tensorflow.keras import layers
from tensorflow.keras import Model

!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

In [2]:
os.listdir('../../Kaggle/')

['cat_dog_test',
 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5',
 'sarcasm.json',
 'tmp']

In [3]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [4]:
local_weights_file = '../../Kaggle/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [6]:
pre_trained_model = InceptionV3(input_shape=(150,150,3),
                               include_top = False,
                               weights = None)
pre_trained_model.load_weights(local_weights_file)

In [7]:
#pre_trained_model.trainable=False

for layer in pre_trained_model.layers:
    layer.trainable = False


In [ ]:
#pre_trained_model.summary()

In [8]:
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output
print(last_output)

last layer output shape:  (None, 7, 7, 768)
Tensor("mixed7/concat_1:0", shape=(None, 7, 7, 768), dtype=float32)


In [9]:
from tensorflow.keras.optimizers import RMSprop

x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(1, activation='sigmoid')(x)

model = Model( pre_trained_model.input, x)
#model.summary()
model.compile( optimizer= RMSprop(lr=0.0001),
             loss = 'binary_crossentropy',
             metrics=['acc'])

In [10]:
os.listdir('../../Kaggle/tmp/dataset/')

['test_set', 'training_set']

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
base_dir = '../../Kaggle/tmp/dataset/'
train_dir = os.path.join( base_dir, 'training_set')
test_dir = os.path.join( base_dir, 'test_set')

train_datagen = ImageDataGenerator( rescale = 1.0/255.0,
                                  rotation_range = 40,
                                  width_shift_range =0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range = 0.2,
                                  horizontal_flip = True)
test_datagen = ImageDataGenerator( rescale = 1.0/255.0)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                   batch_size=20,
                                                   class_mode = 'binary',
                                                   target_size=(150,150))
test_generator = test_datagen.flow_from_directory(test_dir,
                                                 batch_size=20,
                                                 class_mode='binary',
                                                 target_size=(150,150))

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [12]:
history = model.fit(train_generator,
                    steps_per_epoch=100,
                   validation_data = test_generator,
                   epochs = 20,
                    validation_steps=50,
                   verbose=1)

Epoch 1/20
100/100 [==============================] - 128s 1s/step - loss: 0.3510 - acc: 0.8625 - val_loss: 0.1442 - val_acc: 0.9400
Epoch 2/20
100/100 [==============================] - 127s 1s/step - loss: 0.2342 - acc: 0.9165 - val_loss: 0.1297 - val_acc: 0.9480
Epoch 3/20
100/100 [==============================] - 125s 1s/step - loss: 0.2094 - acc: 0.9230 - val_loss: 0.1064 - val_acc: 0.9550
Epoch 4/20
100/100 [==============================] - 128s 1s/step - loss: 0.1961 - acc: 0.9265 - val_loss: 0.2064 - val_acc: 0.9310
Epoch 5/20
100/100 [==============================] - 128s 1s/step - loss: 0.2091 - acc: 0.9300 - val_loss: 0.1265 - val_acc: 0.9480
Epoch 6/20
100/100 [==============================] - 126s 1s/step - loss: 0.2007 - acc: 0.9325 - val_loss: 0.1398 - val_acc: 0.9530
Epoch 7/20
100/100 [==============================] - 126s 1s/step - loss: 0.1702 - acc: 0.9330 - val_loss: 0.1183 - val_acc: 0.9650
Epoch 8/20
100/100 [==============================] - 125s 1s/step - 

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()